# Collision Percentages

The following code was used to compute the collision percentages cited in the article "Behavioral Diversity Generation in Autonomous Exploration Through Reuse of Past Experience" by Fabien C. Y. Benureau and Pierre-Yves Oudeyer.

It available [there](http://fabien.benureau.com/code/frontiers2016.html) and is distributed under the [Open Science License](http://fabien.benureau.com/openscience.html). For any questions and remarks about the code, contact [fabien.benureau@gmail.com](mailto:fabien.benureau@gmail.com).

In [ ]:
import experiments

import dotdot
from fig13_cluster import rmb_reuse_short
from fig14_cluster import dissimilar
from fig15_cluster import pool


def collisions(expcfg):
    try:
        # only consider random motor babbling
        rmb_steps = expcfg.exploration.explorer.eras[0]
    except KeyError:
        rmb_steps = expcfg.exploration.steps
    
    cols, total = 0, 0
    for i in range(expcfg.exp.repetitions):  
        data = experiments.load_exploration(expcfg, rep=i)
        for s in data['s_signals'][:rmb_steps]:
            total += 1
            if s['push_saliency'] > 0:
                cols += 1
    
    return cols, total


# getting expcfg of relevant source tasks
expcfgs = rmb_reuse_short()
cube_src_cfg = expcfgs[0][1]
ball_src_cfg = expcfgs[1][1]

expcfgs = dissimilar()
displaced_src_cfg = expcfgs[1][1]

expcfgs = pool()
pool_nor_cfg = expcfgs[0][0]

# computing collisions
cube_cols      = collisions(cube_src_cfg)
ball_cols      = collisions(ball_src_cfg)
displaced_cols = collisions(displaced_src_cfg)
pool_cols      = collisions(pool_nor_cfg)

In [ ]:
def print_collisions(name, col_data):
    return '{}: {: 5d}/{} collisions, {:.3f}% probability'.format(
        name, col_data[0], col_data[1], 100.0*col_data[0]/col_data[1])

print(print_collisions('cube     ', cube_cols))
print(print_collisions('ball     ', ball_cols))
print(print_collisions('displaced', displaced_cols))
print(print_collisions('pool     ', pool_cols))

In [ ]:
import math

# some precomputations
logs = [None] + [math.log(k) for k in range(1, 10000)]
logfacts = [0.0]
for k in range(1, 10000):
    logfacts.append(logfacts[-1] + logs[k])
logfacts[0] = 1.0

def nCr(n,r): 
    """Compute n choose r"""
    return math.exp(logfacts[n] - logfacts[r] - logfacts[n-r])

def proba_collision(col_data, steps, n):
    """Return the probability to observe at least n collision over a given number of steps"""
    p_col = 1.0*col_data[0]/col_data[1]
    p = 0
    for k in range(n, steps+1):
        p += nCr(steps, k) * p_col**k * (1-p_col)**(steps - k)
    return p

def print_proba_collision(name, col_data, steps, n=1):
    print('{}: there is{: 6.2f}% to observe at least {} collisions over {} steps'.format(
        name, 100.0*proba_collision(col_data, steps, n=n), n, steps))
    
print_proba_collision('cube     ', cube_cols,      200, n=1)
print_proba_collision('ball     ', ball_cols,      200, n=1)
print_proba_collision('ball     ', ball_cols,      100, n=1)
print_proba_collision('displaced', displaced_cols, 200, n=1)
print_proba_collision('displaced', displaced_cols, 100, n=1)
print_proba_collision('pool     ', pool_cols,      300, n=1)
print('')
print_proba_collision('cube     ', cube_cols,      200, n=2)
print_proba_collision('ball     ', ball_cols,      200, n=2)
print_proba_collision('displaced', displaced_cols, 200, n=2)
print_proba_collision('pool     ', pool_cols,      300, n=2)

## Provenance Data

The cluster provenance code examines all the exploration data files of this experiment to check and compare their embedded provenance data.

In [ ]:
import provenance # this may take a minute or two.
prov_data = provenance.cluster([[cube_src_cfg, ball_src_cfg, displaced_src_cfg, pool_nor_cfg]]) 
print(prov_data.message())